Importar Bibliotecas

In [ ]:
import pandas as pd

Importar base de dados original

In [ ]:
path = "databases/MICRODADOS_ENEM_2023.csv" # 1.735 GB (Excluído)
try:
    df = pd.read_csv(path, sep=";", encoding="latin-1", low_memory=False)
    display(df) # 3933955 rows × 76 columns (3m 19.3s)
except:
    print(f"Arquivo {path} não encontrado.")
    df = pd.read_csv("databases/ENEM_2023_SP_1.csv")

Filtrando estudantes da cidade de São Paulo, que não são treineros e não faltaram ou foram eliminados em nenhuma prova.

In [ ]:
def filter_sp_students(df:pd.DataFrame):
    filter = (
        (df["NO_MUNICIPIO_ESC"] == "São Paulo") & # Estudou na cidade de São Paulo 
        (df["IN_TREINEIRO"] == 0) &               # Não é treineiro
        (df["TP_PRESENCA_CH"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_CN"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_LC"] == 1) &             # Não faltou, nem foi eliminado da prova
        (df["TP_PRESENCA_MT"] == 1)               # Não faltou, nem foi eliminado da prova
    )
    return df[filter]

df = pd.read_csv("databases/ENEM_2023_SP_1.csv")
df = filter_sp_students(df) 
df.to_csv("databases/ENEM_2023_SP_1.csv", index=False)

display(df) # 35416 rows × 76 columns

Excluindo colunas "desnecessárias", mas mantendo aquelas que serão utilizadas para calcular outros campos. Também feita a renomeação de algumas colunas.

In [ ]:
def filter_columns(df:pd.DataFrame):
    cols = ["TP_FAIXA_ETARIA", "TP_SEXO", "TP_COR_RACA", "TP_ENSINO", "TP_DEPENDENCIA_ADM_ESC", "NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO", "Q001", "Q002", "Q005", "Q006", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", "Q016", "Q017", "Q019", "Q022", "Q024", "Q025",]
    df = df[cols]

    rename_dict = {
        "Q001": "ESCOLARIDADE_PAI",
        "Q002": "ESCOLARIDADE_MAE",
        "Q005": "N_PESSOAS_CASA",
        "Q006": "RENDA_MENSAL_CASA",
        "Q010": "N_CARRO_CASA",
        "Q011": "N_MOTO_CASA",

        "Q012": "N_GELADEIRA_CASA",
        "Q013": "N_FREEZER_CASA",
        "Q014": "N_LAVA_ROUPA_CASA",
        "Q015": "N_SECA_ROUPA_CASA",
        "Q016": "N_MICROONDAS_CASA",
        "Q017": "N_LAVA_LOUCA_CASA",
        "Q019": "N_TV_CASA",

        "Q022": "N_CELULAR_CASA",
        "Q024": "N_COMP_CASA",
        "Q025": "INTERNET_CASA"
    }

    return df.rename(columns=rename_dict)

df = pd.read_csv("databases/ENEM_2023_SP_1.csv")
df = filter_columns(df)
df.to_csv("databases/ENEM_2023_SP_2.csv", index=False)

display(df) # 35416 rows × 26 columns

Sequência de value counts para observar proporções de certas categorias na amostra.

In [ ]:
def all_value_counts(df:pd.DataFrame):
    categoric_cols = ["TP_FAIXA_ETARIA", "TP_SEXO", "TP_COR_RACA", "TP_ENSINO", "TP_DEPENDENCIA_ADM_ESC", "ESCOLARIDADE_PAI", "ESCOLARIDADE_MAE", "N_PESSOAS_CASA", "RENDA_MENSAL_CASA", "N_GELADEIRA_CASA", "N_FREEZER_CASA", "N_LAVA_ROUPA_CASA", "N_SECA_ROUPA_CASA", "N_MICROONDAS_CASA", "N_LAVA_LOUCA_CASA", "N_TV_CASA", "N_CARRO_CASA", "N_MOTO_CASA", "N_CELULAR_CASA", "N_COMP_CASA", "INTERNET_CASA"]
    for col in categoric_cols:
        print(col)
        vc_df = df[col].value_counts(dropna=False).reset_index()
        vc_df.columns = ['VALOR', 'FREQUENCIA']
        display(vc_df)

df = pd.read_csv("databases/ENEM_2023_SP_2.csv")
all_value_counts(df)

In [ ]:
def drop_not_available_data(df:pd.DataFrame):
    filter = (
        (df["TP_COR_RACA"] == 0) |
        (df["TP_COR_RACA"] == 6) |
        (df["TP_ENSINO"] != 1.0) |
        (df["ESCOLARIDADE_MAE"] == "H") | 
        (df["ESCOLARIDADE_PAI"] == "H")
    )

    df = df.drop(columns=["TP_ENSINO"])
    return df.loc[~filter].reset_index(drop=True)

df = pd.read_csv("databases/ENEM_2023_SP_2.csv")
df = drop_not_available_data(df)
df.to_csv("databases/ENEM_2023_SP_3.csv", index=False)

display(df) # 31731 rows × 25 columns (3703 linhas escluídas)

Obtendo colunas de estimação para renda, faixa etária, além de outra para indicar a estimativa de carros, eletrodomésticos, computadores e celular por pessoa da família.

In [ ]:
def create_estimative_and_simplified_columns(df:pd.DataFrame):
    map_renda = {
        "A": 0,                       # Nenhuma renda
        "B": (0 + 1320)/2,            # Até 1320 (médio = 660)
        "C": (1320.01 + 1980)/2,      # 1650
        "D": (1980.01 + 2640)/2,      # 2310
        "E": (2640.01 + 3300)/2,      # 2970
        "F": (3300.01 + 3960)/2,      # 3630
        "G": (3960.01 + 5280)/2,      # 4620
        "H": (5280.01 + 6600)/2,      # 5940
        "I": (6600.01 + 7920)/2,      # 7260
        "J": (7920.01 + 9240)/2,      # 8580
        "K": (9240.01 + 10560)/2,     # 9900
        "L": (10560.01 + 11880)/2,    # 11220
        "M": (11880.01 + 13200)/2,    # 12540
        "N": (13200.01 + 15840)/2,    # 14520
        "O": (15840.01 + 19800)/2,    # 17820
        "P": (19800.01 + 26400)/2,    # 23100
        "Q": (26400.01 + 33000)/2     # Arbitrário, assume R$ 33000 pois é a mesma distância do intervalo anterior
    }
    map_idade = {
        1: 16,               # Menor de 17 anos -> assume 16
        2: 17,
        3: 18,
        4: 19,
        5: 20,
        6: 21,
        7: 22,
        8: 23,
        9: 24,
        10: 25,
        11: (26 + 30)/2,   # 28
        12: (31 + 35)/2,   # 33
        13: (36 + 40)/2,   # 38
        14: (41 + 45)/2,   # 43
        15: (46 + 50)/2,   # 48
        16: (51 + 55)/2,   # 53
        17: (56 + 60)/2,   # 58
        18: (61 + 65)/2,   # 63
        19: (66 + 70)/2,   # 68
        20: (71 + 75)/2    # Maior de 70 anos
    }
    map_quant = {
        "A": 0,
        "B": 1,
        "C": 2,
        "D": 3,
        "E": 4.5
    }
    cols_to_map = ["N_CELULAR_CASA", "N_COMP_CASA", "N_CARRO_CASA", "N_MOTO_CASA", "N_GELADEIRA_CASA", "N_FREEZER_CASA", "N_LAVA_ROUPA_CASA", "N_SECA_ROUPA_CASA", "N_MICROONDAS_CASA", "N_LAVA_LOUCA_CASA", "N_TV_CASA"]

    df[cols_to_map] = df[cols_to_map].apply(lambda x: x.map(map_quant))

    df["EST_IDADE"] = df["TP_FAIXA_ETARIA"].map(map_idade)
    df["EST_RENDA_PER_CAP"] = (df["RENDA_MENSAL_CASA"].map(map_renda))/df["N_PESSOAS_CASA"]
    df["EST_CELULAR_PER_CAP"] = (df["N_CELULAR_CASA"])/df["N_PESSOAS_CASA"]
    df["EST_COMP_PER_CAP"] = (df["N_COMP_CASA"])/df["N_PESSOAS_CASA"]
    df["EST_VEICULO_PER_CAP"] = (df[["N_CARRO_CASA", "N_MOTO_CASA"]].sum(axis=1))/df["N_PESSOAS_CASA"]
    df["EST_ELE_DOM_PER_CAP"] = (df[["N_GELADEIRA_CASA", "N_FREEZER_CASA", "N_LAVA_ROUPA_CASA", "N_SECA_ROUPA_CASA", "N_MICROONDAS_CASA", "N_LAVA_LOUCA_CASA", "N_TV_CASA"]].sum(axis=1))/df["N_PESSOAS_CASA"]

    df = df.drop(columns=cols_to_map)
    df = df.drop(columns=["TP_FAIXA_ETARIA", "N_PESSOAS_CASA", "RENDA_MENSAL_CASA"])
    return df

df = pd.read_csv("databases/ENEM_2023_SP_3.csv")
df = create_estimative_and_simplified_columns(df)
df.to_csv("databases/ENEM_2023_SP_4.csv", index=False)

display(df) # 31731 rows × 17 columns


Detectando e analisando outliers (usando o método Tukey/IQR)

In [ ]:
def overview_outliers_for_column(df:pd.DataFrame, col:str) -> dict:    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)

    inf = Q1 - 1.5 * (Q3 - Q1)
    sup = Q3 + 1.5 * (Q3 - Q1)
    outliers_inf = df[df[col] < inf].shape[0]
    outliers_sup = df[df[col] > sup].shape[0]

    return {
        "COLUNA": col,
        "LIMITE_INF": inf,
        "LIMITE_SUP": sup,
        "N_OUTLIERS_INF": outliers_inf,
        "N_OUTLIERS_SUP": outliers_sup,
        "TOTAL_OUTLIERS": outliers_inf + outliers_sup
    }

def overview_outliers(df:pd.DataFrame):
    cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "EST_IDADE", "EST_RENDA_PER_CAP", "EST_CELULAR_PER_CAP", "EST_COMP_PER_CAP", "EST_VEICULO_PER_CAP", "EST_ELE_DOM_PER_CAP"]
    results = [overview_outliers_for_column(df, col) for col in cols]
    return pd.DataFrame(results)

df = pd.read_csv("databases/ENEM_2023_SP_4.csv")
outliers = overview_outliers(df)

display(outliers)

Comparando métricas dos outliers com a amostra total

In [ ]:
def get_outliers(df:pd.DataFrame, col:str):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    inf = Q1 - 1.5 * (Q3 - Q1)
    sup = Q3 + 1.5 * (Q3 - Q1)

    return df[(df[col] < inf) | (df[col] > sup)]

def get_no_outliers(df:pd.DataFrame, col:str):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    inf = Q1 - 1.5 * (Q3 - Q1)
    sup = Q3 + 1.5 * (Q3 - Q1)

    return df[(df[col] > inf) & (df[col] < sup)]

def compare_outliers(df:pd.DataFrame, outliers_col:str, comp_col: str):
    df_outliers = get_outliers(df, outliers_col)
    df_no_outliers = get_no_outliers(df, outliers_col)
    stats_df_no_outliers = {
        "Média": df_no_outliers[comp_col].mean(),
        "Mediana": df_no_outliers[comp_col].median(),
        "Variância": df_no_outliers[comp_col].var(),
        "Desvio Padrão": df_no_outliers[comp_col].std(),
        "Amplitude": df_no_outliers[comp_col].max() - df_no_outliers[comp_col].min()
    }
    stats_df_outliets = {
        "Média": df_outliers[comp_col].mean(),
        "Mediana": df_outliers[comp_col].median(),
        "Variância": df_outliers[comp_col].var(),
        "Desvio Padrão": df_outliers[comp_col].std(),
        "Amplitude": df_outliers[comp_col].max() - df_outliers[comp_col].min()
    }

    return pd.DataFrame({"NÃO OUTLIERS": stats_df_no_outliers, "OUTLIERS": stats_df_outliets})

df = pd.read_csv("databases/ENEM_2023_SP_4.csv")
cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO"]
for col in cols:
    print(col)
    comp = compare_outliers(df, "EST_IDADE", col)
    display(comp)

Gerar recorte aleatório de 10000 linhas

In [ ]:
def generate_sample(df:pd.DataFrame, sample_size:int):
    return df.sample(sample_size)

df = pd.read_csv("databases/ENEM_2023_SP_4.csv")
sample = generate_sample(df, 10000)
#sample.to_csv("databases/ENEM_2023_SP_5.csv", index=False)

display(sample) # 10000 rows x 17 columns

Comparando recorte aleatório com sua amostra geradora

In [ ]:
def compare_numeric_col(df:pd.DataFrame, sample:pd.DataFrame, col:str):
    stats_df = {
        "Média": df[col].mean(),
        "Mediana": df[col].median(),
        "Variância": df[col].var(),
        "Desvio Padrão": df[col].std(),
    }
    stats_sample = {
        "Média": sample[col].mean(),
        "Mediana": sample[col].median(),
        "Variância": sample[col].var(),
        "Desvio Padrão": sample[col].std(),
    }

    comp = pd.DataFrame({"TUDO": stats_df, "AMOSTRA": stats_sample})
    comp["DESVIO"] = (comp["AMOSTRA"] - comp["TUDO"])/comp["TUDO"]
    return comp

def compare_categoric_cols(df:pd.DataFrame, sample:pd.DataFrame, col:str):
    total_pct = df[col].value_counts(normalize=True)
    sample_pct = sample[col].value_counts(normalize=True)
    comp = pd.concat([total_pct, sample_pct], axis=1).fillna(0)

    comp.columns = ['TUDO', 'AMOSTRA']
    comp['DESVIO'] = (comp['AMOSTRA'] - comp['TUDO']) / comp['TUDO']

    return comp

def compare_sample(df:pd.DataFrame, sample:pd.DataFrame):
    numeric_cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "EST_IDADE", "EST_RENDA_PER_CAP", "EST_VEICULO_PER_CAP",	"EST_CELULAR_PER_CAP", "EST_COMP_PER_CAP", "EST_ELE_DOM_PER_CAP"]
    categoric_cols = ["TP_SEXO", "TP_COR_RACA",	"TP_DEPENDENCIA_ADM_ESC", "ESCOLARIDADE_PAI",	"ESCOLARIDADE_MAE",	"INTERNET_CASA"]

    for col in numeric_cols:
        print(col)
        display(compare_numeric_col(df, sample, col))

    for col in categoric_cols:
        print(col)
        display(compare_categoric_cols(df, sample, col))

df = pd.read_csv("databases/ENEM_2023_SP_4.csv")
sample = pd.read_csv("databases/ENEM_2023_SP_5.csv")
compare_sample(df, sample)

Alterando nome das categorias para torná-las mais descritivas

In [ ]:
def rename_values(df:pd.DataFrame):
    map_sexo = {
        "M": "Masculino",
        "F": "Feminino"
    }
    map_cor_raca = {
        0: "Nao declarado",
        1: "Branca",
        2: "Preta",
        3: "Parda",
        4: "Amarela",
        5: "Indigena",
        6: "Nao dispõe da informação"
    }
    map_dependencia = {
        1: "Federal",
        2: "Estadual",
        3: "Municipal",
        4: "Privada"
    }
    map_escolaridade = {
        "A": "Nunca estudou",
        "B": "EF-1 incompleto",      # até 4ª série/5º ano
        "C": "EF-2 incompleto",        # completou 4ª, não completou 8ª
        "D": "EM incompleto",
        "E": "ES incompleto",
        "F": "PG incompleto",
        "G": "PG completo",
        "H": "Indisponivel"
    }
    map_internet = {
        "A": False,
        "B": True
    }
    
    df["TP_SEXO"] = df["TP_SEXO"].map(map_sexo)
    df["TP_COR_RACA"] = df["TP_COR_RACA"].map(map_cor_raca)
    df["TP_DEPENDENCIA_ADM_ESC"] = df["TP_DEPENDENCIA_ADM_ESC"].map(map_dependencia)
    df["ESCOLARIDADE_MAE"] = df["ESCOLARIDADE_MAE"].map(map_escolaridade)
    df["ESCOLARIDADE_PAI"] = df["ESCOLARIDADE_PAI"].map(map_escolaridade)
    df["INTERNET_CASA"] = df["INTERNET_CASA"].map(map_internet)

    return df

df = pd.read_csv("databases/ENEM_2023_SP_5.csv")
df = rename_values(df)
df.to_csv("databases/ENEM_2023_FINAL.csv", index=False)

display(df)